In [1]:
from google.cloud import bigquery
from pydantic import BaseModel
from sqlalchemy import create_engine
import pandas as pd


In [2]:
def list_column_names(project_id, dataset_id, table_id):
    client = bigquery.Client(project=project_id)
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    
    # Get the table schema
    table = client.get_table(table_ref)
    
    # Extract and return the column names
    column_names = [field.name for field in table.schema]
    
    filtered_categories = [category for category in column_names if "Adult" in category and "Promotion Description" not in category]
    
    return filtered_categories

# Usage example
PROJECT = "wagon-bootcamp-377120"
DATASET = "g_adventures_dataset"
TABLE = "one_month"

column_names = list_column_names(PROJECT, DATASET, TABLE)
print(column_names)

['Deluxe___Double___D___Adult', 'Standard___Twin___D___Adult', 'Standard___Adult', 'En_suite_bathroom___Twin___D___Adult', 'Shared_bathroom___Twin___D___Adult', 'En_suite_bathroom___Double___D___Adult', 'Upper_Deck_Balcony___Single___D___Adult', 'Main_Deck___Double___D___Adult', 'Upper_Deck_Balcony___Double_Twin___D___Adult', 'Main_Deck_Balcony___Double_Twin___D___Adult', 'Shared_bathroom___Female_only_dorm_____F___Adult', 'Shared_bathroom___Double_Twin___D___Adult', 'Shared_bathroom___Mixed_gender_dorm_____B___Adult', 'Standard___Double___D___Adult', 'Upper_Deck___Twin___D___Adult', 'Non_View_En_suite_bathroom___Twin___D___Adult', 'View_En_suite_bathroom___Twin___D___Adult', 'Non_View_En_suite_bathroom___Double___D___Adult', 'Main_Deck___Twin___D___Adult', 'Lower_Deck___Twin___D___Adult']


In [3]:
class UserTravelDetails(BaseModel):
    country: str
    max_budget: float
    min_budget: float
    departing_after: str
    departing_before: str
    max_duration: int
    min_duration: int

In [4]:
# Example usage:
user_travel_details = UserTravelDetails(
    country="Thailand",
    max_budget=1000,
    min_budget=0,
    departing_after="2024-04-01",
    departing_before="2024-04-15",
    max_duration=10,
    min_duration=5
)

In [5]:
def generate_sql_query(room_categories, user_travel_details):
    category_cases = []

    for category in room_categories:
        category_case = f"CASE WHEN {category} > 0 THEN {category} ELSE 1000000 END"
        category_cases.append(category_case)

    category_cases_str = ",\n".join(category_cases)

    query = f"""
    SELECT *
FROM (
    SELECT
        tour_name,
        itinerary_name,
        visited_countries,
        start_date,
        duration,
        LEAST(
            {category_cases_str}
        ) AS cost
    FROM {PROJECT}.{DATASET}.{TABLE}
) AS subquery
    WHERE 1 = 1
    """

    
    # Iterate through the provided filter criteria and add them to the query
    if user_travel_details.country:
        query += f" AND visited_countries LIKE '%{user_travel_details.country}%'"

    if user_travel_details.max_budget:
        query += f" AND cost <= {user_travel_details.max_budget}"

    if user_travel_details.min_budget:
        query += f" AND cost >= {user_travel_details.min_budget}"

    if user_travel_details.departing_after:
        query += f" AND start_date >= '{user_travel_details.departing_after}'"

    if user_travel_details.departing_before:
        query += f" AND start_date <= '{user_travel_details.departing_before}'"

    if user_travel_details.max_duration:
        query += f" AND duration <= {user_travel_details.max_duration}"

    if user_travel_details.min_duration:
        query += f" AND duration >= {user_travel_details.min_duration}"
    
    return query

In [6]:
from google.cloud import bigquery

PROJECT = "wagon-bootcamp-377120"
DATASET = "g_adventures_dataset"
TABLE = "one_month"

query = generate_sql_query(column_names, user_travel_details)

client = bigquery.Client(project="wagon-bootcamp-377120")
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()


In [7]:
df

,tour_name,itinerary_name,visited_countries,start_date,duration,cost
0,"Cambodia to Bangkok: Phnom Penh, Siem Reap & K...",None,"Cambodia, Thailand",2024-04-14,5,299.0
1,"Thailand Adventure: Bangkok, Chiang Mai & Stre...",None,Thailand,2024-04-07,7,439.0
2,"Bangkok to Hanoi: Chiang Mai, Night Markets & ...",None,"Vietnam, Thailand",2024-04-07,9,729.0
3,Northern Thailand Hilltribes Trek,None,Thailand,2024-04-11,5,319.0
4,Northern Thailand Hilltribes Trek,None,Thailand,2024-04-04,5,319.0
5,Northern Thailand Adventure,None,Thailand,2024-04-06,10,929.0
6,Cambodia Experience,None,"Vietnam, Cambodia, Thailand",2024-04-04,9,739.0
7,Cambodia Experience,None,"Vietnam, Cambodia, Thailand",2024-04-08,9,739.0
8,Cambodia Experience,None,"Vietnam, Cambodia, Thailand",2024-04-11,9,739.0
9,Cambodia Experience,None,"Vietnam, Cambodia, Thailand",2024-04-14,9,739.0
